# SageSaver MongoDB Connection Example
Congrats on setting up your SageSaver Notebook! This notebook will guide you through connecting to your Mongo Database

## 1 - Specify Environment Stack Name
This Notebook Server has stored some information on your SageSaver environment. Double-check that the environment stack name below is correct.

In [45]:
from config import config
ENVIRONMENT_STACK_NAME = config["aws"]["stack-origin"]
AWS_REGION = config["aws"]["region"]

print("Environment Stack Name:", ENVIRONMENT_STACK_NAME)

Environment Stack Name: Secrets2


## 2 - Retrieve Secret from AWS
Your database credentials have been stored in AWS Secret Manager. The credential is named ENVIRONMENT_STACK_NAME-Database-Secret and you can reuse the methods below to retrieve them.

In [ ]:
import json
import boto3

def get_database_secret(client, stack):
    '''Retrieves SageSaver database credentials from Secrets Manager

    Args:
        client (boto3 service client): Secrets Manager service client
            used for credential retrieval
        stack (string): Name of the SageSaver Environment Stack housing
             the database
    
    Returns:
        json: Database credentials
    '''

    secret_name = f'{stack}-Database-Secret'
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    secret = get_secret_value_response['SecretString']

    return json.loads(secret)

def get_mongo_client(session, stack):
    '''Generates a mongo client to the SageSaver MongoDB database

    Args:
        session (boto3 session): Session used to generate a
            Secrets Manager client
        stack (string): Name of the SageSaver Environment Stack housing
            the database
    
    Returns:
        pymongo client: Client connected to the SageSaver database
    '''

    secrets_client = session.client('secretsmanager')
    secret = get_database_secret(secrets_client, stack)

    db_username = secret['username']
    db_password = secret['password']
    db_port = secret['port']
    db_host = secret['host']

    return MongoClient(
        username=db_username,
        password=db_password,
        port=db_port,
        host=db_host
    )

aws_session = boto3.session.Session(region_name = AWS_REGION)
client = get_mongo_client(aws_session, STACK_NAME)

## 3 - Test Your Database Connection
At this point, 'client' is fully connected to your SageSaver MongoDB server! Feel free to perform any MongoDB/pymongo actions below:

In [48]:
print(client.list_database_names())

['admin', 'config', 'local', 'osu_db', 'osu_random_feb_2020']
